In [1]:
import json
from xmitgcm import llcreader
from os.path import abspath, expanduser, expandvars
from os import PathLike
from typing import List, Union
import dask
import xarray as xr

In [2]:
model = llcreader.PleiadesLLC4320Model()

In [3]:
klevel = list(range(90))
ds = model.get_dataset(varnames=['Eta'], read_grid=True)

/srv/conda/envs/notebook/lib/python3.10/site-packages/xmitgcm/llcreader/llcmodel.py:406: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  return {vname: _facets_to_faces(data, nfaces)
/srv/conda/envs/notebook/lib/python3.10/site-packages/xmitgcm/llcreader/llcmodel.py:406: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the l

In [13]:
ds_isel = ds.drop_vars(['Eta','dxF','dyF','dxV','dyU']).isel(k=0,k_u=0,k_l=0,k_p1=0)#.reset_coords()
vars_to_chunk = ds_isel.drop_vars(['i','face','i_g','j','j_g','k','k_u','k_l','k_p1','niter',
              'time','drC','drF','PHrefC','PHrefF','Z','Zp1','rhoRef','Zl','Zu'])
coords_to_chunk = vars_to_chunk.coords

In [15]:


comp = dict(zlib=True, complevel=1, chunksizes=((1,360,360)))
#encoding = {var: comp for var in vars_to_chunk}
encoding = {var: comp for var in coords_to_chunk}

In [16]:
for faceno in range(0,13):
    path = 'llc4320_grid_f' + str(faceno) +'.nc'
    ds_face = ds_isel.isel(face=[faceno])
    ds_face.to_netcdf(path,mode="w",encoding=encoding)

In [17]:
import os
arr = arr = [filename for filename in os.listdir('.') if filename.startswith('llc4320_grid')]
urls = ["/nobackup/csjone15/pleiades_llc_recipes/python_cli_data_export/" + p for p in arr]

In [18]:
import xarray as xr
import kerchunk.hdf
import fsspec
import ujson

In [19]:
so = dict(
    anon=True, default_fill_cache=False, default_cache_type='first'
)
singles = []
fs2 = fsspec.filesystem('')
for u in urls:
    print(u)
    with fsspec.open(u, **so) as inf:
        h5chunks = kerchunk.hdf.SingleHdf5ToZarr(inf, u, inline_threshold=100)
        #singles.append(h5chunks.translate())
        ##
        #variable = u.split('/')[-1].split('.')[0]
        #iter = u.split('/')[-1].split('.')[0].split('_')[-1]
        outpath = f'/nobackup/csjone15/pleiades_llc_recipes/python_cli_data_export/grid/'
        filename = u.split('/')[-1].split('.')[0] + '.json'
        outf = outpath + filename #file name to save json to
        print(outf)
        with fs2.open(outf, 'wb') as f:
            f.write(ujson.dumps(h5chunks.translate()).encode());

/nobackup/csjone15/pleiades_llc_recipes/python_cli_data_export/llc4320_grid_f11.nc
/nobackup/csjone15/pleiades_llc_recipes/python_cli_data_export/grid/llc4320_grid_f11.json
/nobackup/csjone15/pleiades_llc_recipes/python_cli_data_export/llc4320_grid_f8.nc
/nobackup/csjone15/pleiades_llc_recipes/python_cli_data_export/grid/llc4320_grid_f8.json
/nobackup/csjone15/pleiades_llc_recipes/python_cli_data_export/llc4320_grid_f0.nc
/nobackup/csjone15/pleiades_llc_recipes/python_cli_data_export/grid/llc4320_grid_f0.json
/nobackup/csjone15/pleiades_llc_recipes/python_cli_data_export/llc4320_grid_f4.nc
/nobackup/csjone15/pleiades_llc_recipes/python_cli_data_export/grid/llc4320_grid_f4.json
/nobackup/csjone15/pleiades_llc_recipes/python_cli_data_export/llc4320_grid_f12.nc
/nobackup/csjone15/pleiades_llc_recipes/python_cli_data_export/grid/llc4320_grid_f12.json
/nobackup/csjone15/pleiades_llc_recipes/python_cli_data_export/llc4320_grid_f7.nc
/nobackup/csjone15/pleiades_llc_recipes/python_cli_data_exp

In [19]:
test = xr.open_dataset('llc4320_grid_f0.nc',chunks={})

FileNotFoundError: [Errno 2] No such file or directory: b'/nobackupp12/csjone15/pleiades_llc_recipes/python_cli_data_export/llc4320_grid_f0.nc'

In [15]:
test.hFacC.encoding

{'zlib': True,
 'szip': False,
 'zstd': False,
 'bzip2': False,
 'blosc': False,
 'shuffle': True,
 'complevel': 1,
 'fletcher32': False,
 'contiguous': False,
 'chunksizes': (1, 360, 360),
 'source': '/nobackupp12/csjone15/pleiades_llc_recipes/python_cli_data_export/llc4320_grid.nc',
 'original_shape': (13, 4320, 4320),
 'dtype': dtype('float32'),
 '_FillValue': nan}

In [14]:
ds_isel.encoding["coordinates"] = 

{}

In [25]:
!mkdir grid

In [30]:
pwd

'/nobackupp12/csjone15/pleiades_llc_recipes/python_cli_data_export'